# Feature Creation - V1

This is the first stage of the feature creation process for my model. In this stage I will creating features for every team that gives much more insight into the strength of each teams offense and defense. In addition it provides various other team strength and situational metrics that will be useful for our model later on.

I found that each feature created could be done a similar way, so mosltly this was done by copying code from one cell to the next but changing a few of the naming conventions to correlate to the feature to which I was creating.

I am assuming this was the long way to do this and I am sure there is a much cleaner/efficient way to do this, however this is what I came up with.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('/Users/epainter/Desktop/bet_model_v2/data/clean/master.csv')

In [4]:
df

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,C_fav,CB_fav,DI_fav,Edge_fav,LB_fav,LG_fav,LT_fav,QB_fav,RB_fav,RG_fav,RT_fav,S_fav,TE_fav,WR_fav,C_und,CB_und,DI_und,Edge_und,LB_und,LG_und,LT_und,QB_und,RB_und,RG_und,RT_und,S_und,TE_und,WR_und,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,team_division_fav,team_division_und
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,66.0,64.27,73.70,55.75,59.30,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.10,74.73,67.3,67.30,65.53,75.35,67.20,59.1,75.1,82.4,74.05,53.4,59.4,69.60,56.60,71.07,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,AFC West,AFC South
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,72.1,60.30,79.53,69.50,69.60,87.7,79.7,75.8,62.95,66.6,53.0,65.35,73.10,77.27,56.9,79.33,65.10,59.60,75.60,60.3,74.1,92.0,77.90,79.3,63.6,68.10,73.25,73.63,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,NFC South,NFC West
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,68.7,71.93,74.53,66.85,78.60,63.8,89.4,90.1,81.25,63.6,73.6,73.35,88.50,63.37,72.0,65.47,71.60,83.45,59.67,74.2,88.7,73.5,82.70,56.7,80.5,68.10,72.30,73.45,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,AFC North,AFC North
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,67.3,60.20,61.85,68.50,57.03,55.8,73.1,64.1,80.30,63.1,51.8,75.85,60.20,75.43,72.0,69.73,68.17,70.00,73.75,56.1,83.0,63.2,73.80,65.6,60.6,81.25,68.15,75.90,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,AFC East,AFC East
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,71.0,59.33,68.45,63.85,65.70,74.0,64.9,79.9,80.05,61.8,69.1,73.60,83.20,72.33,63.4,59.70,87.05,74.20,58.37,58.6,79.3,73.0,80.60,63.6,76.2,77.53,51.80,71.20,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,AFC West,NFC South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,Los Angeles Chargers,Kansas City Chiefs,1,12,13,-3.5,35.0,0,0.0,0,18,2023,74.0,58.37,57.30,76.67,53.60,69.8,84.0,77.9,71.10,62.2,59.8,68.05,70.60,79.83,89.9,71.27,64.65,64.83,72.70,78.4,59.3,92.0,66.85,72.3,59.5,64.90,91.30,70.17,Kansas City Chiefs,Los Angeles Chargers,13.0,12.0,268.0,1.0,353.0,1.0,AFC West,AFC West
1067,Philadelphia Eagles,New York Giants,0,10,27,-5.0,42.5,0,0.0,0,18,2023,89.4,72.33,76.25,86.07,69.45,72.1,81.7,88.2,70.80,72.7,84.8,74.25,78.20,77.57,92.3,73.70,82.90,62.90,75.50,57.5,90.3,75.8,74.90,65.6,41.8,61.45,72.40,71.37,New York Giants,Philadelphia Eagles,27.0,10.0,415.0,1.0,299.0,4.0,NFC East,NFC East
1068,San Francisco 49ers,Los Angeles Rams,1,20,21,-5.5,40.0,0,0.0,1,18,2023,62.9,73.30,71.75,72.83,82.65,59.5,91.7,77.7,83.05,51.8,69.4,70.90,84.70,73.63,63.8,54.13,78.90,61.83,70.30,71.0,64.1,67.0,73.95,60.0,73.2,61.65,62.70,70.63,Los Angeles Rams,San Francisco 49ers,21.0,20.0,258.0,1.0,300.0,1.0,NFC West,NFC West
1069,Dallas Cowboys,Washington Commanders,0,38,10,-13.0,47.5,1,1.0,1,18,2023,60.5,70.90,74.05,85.80,69.30,73.3,60.9,71.8,75.20,72.8,73.9,74.05,66.10,73.70,60.0,81.60,71.15,74.87,59.75,43.6,71.6,68.5,79.40,71.6,61.6,74.95,52.00,73.63,Dallas Cowboys,Washington Commanders,38.0,10.0,440.0,1.0,180.0,3.0,NFC East,NFC East


# Offensive Statistics

In [5]:
# Average points scored by each team across each week in a given season

def calculate_running_average(df):
    # Create a dictionary to store running totals and game counts for each team
    team_stats = {}
    
    # Create new columns for running averages
    df['fav_running_avg'] = 0.0
    df['und_running_avg'] = 0.0
    
    # Sort the dataframe by week and season
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        fav_score = row['fav_score']
        und_score = row['und_score']
        week = row['week']
        season = row['season']
        
        # Initialize team stats if not exists
        for team in [fav_team, und_team]:
            if team not in team_stats:
                team_stats[team] = {'total_points': 0, 'game_count': 0, 'season': season}
        
        # Reset stats for new season
        if team_stats[fav_team]['season'] != season:
            team_stats[fav_team] = {'total_points': 0, 'game_count': 0, 'season': season}
        if team_stats[und_team]['season'] != season:
            team_stats[und_team] = {'total_points': 0, 'game_count': 0, 'season': season}
        
        # Update stats for favorite team
        team_stats[fav_team]['total_points'] += fav_score
        team_stats[fav_team]['game_count'] += 1
        
        # Update stats for underdog team
        team_stats[und_team]['total_points'] += und_score
        team_stats[und_team]['game_count'] += 1
        
        # Calculate and store running averages
        df.at[index, 'fav_running_avg'] = round(team_stats[fav_team]['total_points']
                                                / team_stats[fav_team]['game_count'], 2)
        df.at[index, 'und_running_avg'] = round(team_stats[und_team]['total_points']
                                                / team_stats[und_team]['game_count'], 2)
    
    return df

df = calculate_running_average(df)

In [6]:
# Average turnovers per week for each team in a given season

def calculate_turnover_running_average(df):
    # Create a dictionary to store running totals and game counts for each team
    team_stats = {}
    
    # Create new columns for turnover running averages
    df['fav_turnover_running_avg'] = 0.0
    df['und_turnover_running_avg'] = 0.0
    
    # Sort the dataframe by week and season
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        loser = row['loser']
        to_w = row['to_w']
        to_l = row['to_l']
        week = row['week']
        season = row['season']
        
        # Determine which team (favorite or underdog) is the winner and loser
        if fav_team == winner:
            fav_turnovers = to_w
            und_turnovers = to_l
        else:
            fav_turnovers = to_l
            und_turnovers = to_w
        
        # Initialize team stats if not exists
        for team in [fav_team, und_team]:
            if team not in team_stats:
                team_stats[team] = {'total_turnovers': 0, 'game_count': 0, 'season': season}
        
        # Reset stats for new season
        if team_stats[fav_team]['season'] != season:
            team_stats[fav_team] = {'total_turnovers': 0, 'game_count': 0, 'season': season}
        if team_stats[und_team]['season'] != season:
            team_stats[und_team] = {'total_turnovers': 0, 'game_count': 0, 'season': season}
        
        # Update stats for favorite team
        team_stats[fav_team]['total_turnovers'] += fav_turnovers
        team_stats[fav_team]['game_count'] += 1
        
        # Update stats for underdog team
        team_stats[und_team]['total_turnovers'] += und_turnovers
        team_stats[und_team]['game_count'] += 1
        
        # Calculate and store running averages
        if team_stats[fav_team]['game_count'] > 0:
            df.at[index, 'fav_turnover_running_avg'] = round(team_stats[fav_team]['total_turnovers'] 
                                                     / team_stats[fav_team]['game_count'], 2)
        else:
            df.at[index, 'fav_turnover_running_avg'] = fav_turnovers  # or any other default value

        if team_stats[und_team]['game_count'] > 0:
            df.at[index, 'und_turnover_running_avg'] = round(team_stats[und_team]['total_turnovers'] 
                                                     / team_stats[und_team]['game_count'], 2)
        else:
            df.at[index, 'und_turnover_running_avg'] = und_turnovers  # or any other default value
    
    return df

# Apply the function to your dataframe
df = calculate_turnover_running_average(df)

### Important Note

The original function I had here was giving me some issues, but after some analysis I pinpointed the issue and corrected it. It was giving me Nan values for turnovers at random weeks. 

Below is the corrected code.

In [7]:
# Average turnovers per week for a given team

def calculate_turnover_running_average(df):
    team_stats = {}
    df['fav_turnover_running_avg'] = 0.0
    df['und_turnover_running_avg'] = 0.0
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        try:
            fav_team = row['favorite']
            und_team = row['underdog']
            winner = row['winner']
            loser = row['loser']
            to_w = row['to_w']
            to_l = row['to_l']
            week = row['week']
            season = row['season']
            
            # Check for missing data
            if pd.isna(to_w) or pd.isna(to_l):
                print(f"Missing turnover data in week {week}, season {season}")
                continue
            
            # Determine which team is the winner and loser
            if fav_team == winner:
                fav_turnovers = to_w
                und_turnovers = to_l
            elif und_team == winner:
                fav_turnovers = to_l
                und_turnovers = to_w
            else:
                print(f"Winner mismatch in week {week}, season {season}")
                continue
            
            # Initialize team stats if not exists
            for team in [fav_team, und_team]:
                if team not in team_stats:
                    team_stats[team] = {'total_turnovers': 0, 'game_count': 0, 'season': season}
            
            # Reset stats for new season
            if team_stats[fav_team]['season'] != season:
                team_stats[fav_team] = {'total_turnovers': 0, 'game_count': 0, 'season': season}
            if team_stats[und_team]['season'] != season:
                team_stats[und_team] = {'total_turnovers': 0, 'game_count': 0, 'season': season}
            
            # Update stats
            team_stats[fav_team]['total_turnovers'] += fav_turnovers
            team_stats[fav_team]['game_count'] += 1
            team_stats[und_team]['total_turnovers'] += und_turnovers
            team_stats[und_team]['game_count'] += 1
            
            # Calculate and store running averages
            df.at[index, 'fav_turnover_running_avg'] = round(team_stats[fav_team]['total_turnovers'] / team_stats[fav_team]['game_count'], 2)
            df.at[index, 'und_turnover_running_avg'] = round(team_stats[und_team]['total_turnovers'] / team_stats[und_team]['game_count'], 2)
        
        except Exception as e:
            print(f"Error in week {week}, season {season}: {str(e)}")
            df.at[index, 'fav_turnover_running_avg'] = np.nan
            df.at[index, 'und_turnover_running_avg'] = np.nan
    
    return df

# Apply the function to your dataframe
df = calculate_turnover_running_average(df)

# Check for NaN values after calculation
nan_rows = df[df['fav_turnover_running_avg'].isna() | df['und_turnover_running_avg'].isna()]
print(f"Number of rows with NaN values: {len(nan_rows)}")
if len(nan_rows) > 0:
    print(nan_rows[['season', 'week', 'favorite', 'underdog', 'winner', 'loser', 'to_w', 'to_l', 'fav_turnover_running_avg', 'und_turnover_running_avg']])

Missing turnover data in week 3, season 2020
Missing turnover data in week 10, season 2021
Missing turnover data in week 13, season 2022
Number of rows with NaN values: 0


## Important Note

After running this entire script, I was getting a ton of Nan values in various columnns. After doing some analysis, I found out where the issues was coming from.

It was best to address that issue at this point before creating anymore features. Essentially I did some error correcting and decided to do it here (if this does not seem intuitive to reader)

In [8]:
df.isna().sum()

favorite                    0
underdog                    0
fav_home                    0
fav_score                   0
und_score                   0
spread                      0
ou_value                    0
fav_ml_result               0
fav_sp_result               0
ou_result                   0
week                        0
season                      0
C_fav                       0
CB_fav                      0
DI_fav                      0
Edge_fav                    0
LB_fav                      0
LG_fav                      0
LT_fav                      0
QB_fav                      0
RB_fav                      0
RG_fav                      0
RT_fav                      0
S_fav                       0
TE_fav                      0
WR_fav                      0
C_und                       0
CB_und                      0
DI_und                      0
Edge_und                    0
LB_und                      0
LG_und                      0
LT_und                      0
QB_und    

In [9]:
# Imputing correct data here

# Index 39

df.loc[39, 'pts_w'] = 23
df.loc[39, 'pts_l'] = 23
df.loc[39, 'yds_w'] = 304
df.loc[39, 'to_w'] = 0
df.loc[39, 'yds_l'] = 378
df.loc[39, 'to_l'] = 2

# Index 397

df.loc[397, 'pts_w'] = 16
df.loc[397, 'pts_l'] = 16
df.loc[397, 'yds_w'] = 306
df.loc[397, 'to_w'] = 0
df.loc[397, 'yds_l'] = 387
df.loc[397, 'to_l'] = 3

# Index 715

df.loc[715, 'pts_w'] = 20
df.loc[715, 'pts_l'] = 20
df.loc[715, 'yds_w'] = 316
df.loc[715, 'to_w'] = 1
df.loc[715, 'yds_l'] = 411
df.loc[715, 'to_l'] = 1

In [10]:
df.isna().sum()

favorite                    0
underdog                    0
fav_home                    0
fav_score                   0
und_score                   0
spread                      0
ou_value                    0
fav_ml_result               0
fav_sp_result               0
ou_result                   0
week                        0
season                      0
C_fav                       0
CB_fav                      0
DI_fav                      0
Edge_fav                    0
LB_fav                      0
LG_fav                      0
LT_fav                      0
QB_fav                      0
RB_fav                      0
RG_fav                      0
RT_fav                      0
S_fav                       0
TE_fav                      0
WR_fav                      0
C_und                       0
CB_und                      0
DI_und                      0
Edge_und                    0
LB_und                      0
LG_und                      0
LT_und                      0
QB_und    

In [11]:
# Average yards per game (for) a given team in a given season

import pandas as pd

def calculate_yards_running_average(df):
    # Create a dictionary to store running totals and game counts for each team
    team_stats = {}
    
    # Create new columns for yards running averages
    df['fav_yards_running_avg'] = 0.0
    df['und_yards_running_avg'] = 0.0
    
    # Sort the dataframe by week and season
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        loser = row['loser']
        yds_w = row['yds_w']
        yds_l = row['yds_l']
        week = row['week']
        season = row['season']
        
        # Determine which team (favorite or underdog) is the winner and loser
        if fav_team == winner:
            fav_yards = yds_w
            und_yards = yds_l
        else:
            fav_yards = yds_l
            und_yards = yds_w
        
        # Initialize team stats if not exists
        for team in [fav_team, und_team]:
            if team not in team_stats:
                team_stats[team] = {'total_yards': 0, 'game_count': 0, 'season': season}
        
        # Reset stats for new season
        if team_stats[fav_team]['season'] != season:
            team_stats[fav_team] = {'total_yards': 0, 'game_count': 0, 'season': season}
        if team_stats[und_team]['season'] != season:
            team_stats[und_team] = {'total_yards': 0, 'game_count': 0, 'season': season}
        
        # Update stats for favorite team
        team_stats[fav_team]['total_yards'] += fav_yards
        team_stats[fav_team]['game_count'] += 1
        
        # Update stats for underdog team
        team_stats[und_team]['total_yards'] += und_yards
        team_stats[und_team]['game_count'] += 1
        
        # Calculate and store running averages
        df.at[index, 'fav_yards_running_avg'] = round(team_stats[fav_team]['total_yards']
                                                      /team_stats[fav_team]['game_count'], 2)
        df.at[index, 'und_yards_running_avg'] = round(team_stats[und_team]['total_yards']
                                                      / team_stats[und_team]['game_count'], 2)
    
    return df

# Apply the function to your dataframe
df = calculate_yards_running_average(df)

df

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,C_fav,CB_fav,DI_fav,Edge_fav,LB_fav,LG_fav,LT_fav,QB_fav,RB_fav,RG_fav,RT_fav,S_fav,TE_fav,WR_fav,C_und,CB_und,DI_und,Edge_und,LB_und,LG_und,LT_und,QB_und,RB_und,RG_und,RT_und,S_und,TE_und,WR_und,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,team_division_fav,team_division_und,fav_running_avg,und_running_avg,fav_turnover_running_avg,und_turnover_running_avg,fav_yards_running_avg,und_yards_running_avg
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,66.0,64.27,73.70,55.75,59.30,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.10,74.73,67.3,67.30,65.53,75.35,67.20,59.1,75.1,82.4,74.05,53.4,59.4,69.60,56.60,71.07,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,AFC West,AFC South,34.00,20.00,0.00,1.00,369.00,360.00
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,72.1,60.30,79.53,69.50,69.60,87.7,79.7,75.8,62.95,66.6,53.0,65.35,73.10,77.27,56.9,79.33,65.10,59.60,75.60,60.3,74.1,92.0,77.90,79.3,63.6,68.10,73.25,73.63,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,NFC South,NFC West,25.00,38.00,2.00,0.00,506.00,383.00
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,68.7,71.93,74.53,66.85,78.60,63.8,89.4,90.1,81.25,63.6,73.6,73.35,88.50,63.37,72.0,65.47,71.60,83.45,59.67,74.2,88.7,73.5,82.70,56.7,80.5,68.10,72.30,73.45,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,AFC North,AFC North,38.00,6.00,1.00,3.00,381.00,306.00
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,67.3,60.20,61.85,68.50,57.03,55.8,73.1,64.1,80.30,63.1,51.8,75.85,60.20,75.43,72.0,69.73,68.17,70.00,73.75,56.1,83.0,63.2,73.80,65.6,60.6,81.25,68.15,75.90,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,AFC East,AFC East,27.00,17.00,2.00,2.00,404.00,254.00
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,71.0,59.33,68.45,63.85,65.70,74.0,64.9,79.9,80.05,61.8,69.1,73.60,83.20,72.33,63.4,59.70,87.05,74.20,58.37,58.6,79.3,73.0,80.60,63.6,76.2,77.53,51.80,71.20,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,AFC West,NFC South,34.00,30.00,0.00,0.00,372.00,388.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,Los Angeles Chargers,Kansas City Chiefs,1,12,13,-3.5,35.0,0,0.0,0,18,2023,74.0,58.37,57.30,76.67,53.60,69.8,84.0,77.9,71.10,62.2,59.8,68.05,70.60,79.83,89.9,71.27,64.65,64.83,72.70,78.4,59.3,92.0,66.85,72.3,59.5,64.90,91.30,70.17,Kansas City Chiefs,Los Angeles Chargers,13.0,12.0,268.0,1.0,353.0,1.0,AFC West,AFC West,20.35,21.82,1.24,1.65,329.35,351.29
1067,Philadelphia Eagles,New York Giants,0,10,27,-5.0,42.5,0,0.0,0,18,2023,89.4,72.33,76.25,86.07,69.45,72.1,81.7,88.2,70.80,72.7,84.8,74.25,78.20,77.57,92.3,73.70,82.90,62.90,75.50,57.5,90.3,75.8,74.90,65.6,41.8,61.45,72.40,71.37,New York Giants,Philadelphia Eagles,27.0,10.0,415.0,1.0,299.0,4.0,NFC East,NFC East,25.47,15.65,1.65,1.12,354.35,280.00
1068,San Francisco 49ers,Los Angeles Rams,1,20,21,-5.5,40.0,0,0.0,1,18,2023,62.9,73.30,71.75,72.83,82.65,59.5,91.7,77.7,83.05,51.8,69.4,70.90,84.70,73.63,63.8,54.13,78.90,61.83,70.30,71.0,64.1,67.0,73.95,60.0,73.2,61.65,62.70,70.63,Los Angeles Rams,San Francisco 49ers,21.0,20.0,258.0,1.0,300.0,1.0,NFC West,NFC West,28.88,23.76,1.06,1.06,398.41,359.29
1069,Dallas Cowboys,Washington Commanders,0,38,10,-13.0,47.5,1,1.0,1,18,2023,60.5,70.90,74.05,85.80,69.30,73.3,60.9,71.8,75.20,72.8,73.9,74.05,66.10,73.70,60.0,81.60,71.15,74.87,59.75,43.6,71.6,68.5,79.40,71.6,61.6,74.95,52.00,73.63,Dallas Cowboys,Washington Commanders,38.0,10.0,440.0,1.0,180.0,3.0,NFC East,NFC East,29.94,19.35,0.94,1.88,371.59,312.76


In [12]:
# Offensive Rating for both the Favorite and Underdog

# Favorite
df['off_rating_fav'] = (df['C_fav']+df['LG_fav']+df['LT_fav']+
                             df['QB_fav']+df['RB_fav']+df['RG_fav']+
                             df['RT_fav']+df['TE_fav']+df['WR_fav'])/9
# Underdog
df['off_rating_und'] = (df['C_und']+df['LG_und']+df['LT_und']+
                             df['QB_und']+df['RB_und']+df['RG_und']+
                             df['RT_und']+df['TE_und']+df['WR_und'])/9

# Need to round these to 2 decimal places
df['off_rating_fav'] = df['off_rating_fav'].round(2)
df['off_rating_und'] = df['off_rating_und'].round(2)


In [13]:
# There was an issue with the New York Jets Offensive Rating for some reason, so we will fix it here

df['off_rating_fav'] = df['off_rating_fav'].replace(283.93, 72.0)
df['off_rating_und'] = df['off_rating_und'].replace(283.93, 72.0)

# Defensive Statistics

In [14]:
# Yards allowed per game

def calculate_defensive_yards_running_average(df):
    team_stats = {}
    
    df['fav_def_yards_running_avg'] = 0.0
    df['und_def_yards_running_avg'] = 0.0
    
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        loser = row['loser']
        yds_w = row['yds_w']
        yds_l = row['yds_l']
        week = row['week']
        season = row['season']
        
        # Determine yards allowed by each team
        if fav_team == winner:
            fav_yards_allowed = yds_l
            und_yards_allowed = yds_w
        else:
            fav_yards_allowed = yds_w
            und_yards_allowed = yds_l
        
        # Initialize or reset team stats
        for team in [fav_team, und_team]:
            if team not in team_stats or team_stats[team]['season'] != season:
                team_stats[team] = {'total_yards_allowed': 0, 'game_count': 0, 'season': season}
        
        # Update stats
        team_stats[fav_team]['total_yards_allowed'] += fav_yards_allowed
        team_stats[fav_team]['game_count'] += 1
        team_stats[und_team]['total_yards_allowed'] += und_yards_allowed
        team_stats[und_team]['game_count'] += 1
        
        # Calculate and store running averages
        df.at[index, 'fav_def_yards_running_avg'] = round(team_stats[fav_team]['total_yards_allowed'] / team_stats[fav_team]['game_count'], 2)
        df.at[index, 'und_def_yards_running_avg'] = round(team_stats[und_team]['total_yards_allowed'] / team_stats[und_team]['game_count'], 2)
    
    return df

# Apply the function to your dataframe
df = calculate_defensive_yards_running_average(df)

In [15]:
# Turnovers

def calculate_defensive_turnovers_running_average(df):
    team_stats = {}
    
    df['fav_def_turnovers_running_avg'] = 0.0
    df['und_def_turnovers_running_avg'] = 0.0
    
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        loser = row['loser']
        to_w = row['to_w']
        to_l = row['to_l']
        week = row['week']
        season = row['season']
        
        # Determine turnovers forced by each team
        if fav_team == winner:
            fav_turnovers_forced = to_l
            und_turnovers_forced = to_w
        else:
            fav_turnovers_forced = to_w
            und_turnovers_forced = to_l
        
        # Initialize or reset team stats
        for team in [fav_team, und_team]:
            if team not in team_stats or team_stats[team]['season'] != season:
                team_stats[team] = {'total_turnovers_forced': 0, 'game_count': 0, 'season': season}
        
        # Update stats
        team_stats[fav_team]['total_turnovers_forced'] += fav_turnovers_forced
        team_stats[fav_team]['game_count'] += 1
        team_stats[und_team]['total_turnovers_forced'] += und_turnovers_forced
        team_stats[und_team]['game_count'] += 1
        
        # Calculate and store running averages
        df.at[index, 'fav_def_turnovers_running_avg'] = round(team_stats[fav_team]['total_turnovers_forced'] / team_stats[fav_team]['game_count'], 2)
        df.at[index, 'und_def_turnovers_running_avg'] = round(team_stats[und_team]['total_turnovers_forced'] / team_stats[und_team]['game_count'], 2)
    
    return df

# Apply the function to your dataframe
df = calculate_defensive_turnovers_running_average(df)

In [16]:
# Points allowed per game

def calculate_defensive_points_running_average(df):
    team_stats = {}
    
    df['fav_def_points_running_avg'] = 0.0
    df['und_def_points_running_avg'] = 0.0
    
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        loser = row['loser']
        pts_w = row['pts_w']
        pts_l = row['pts_l']
        week = row['week']
        season = row['season']
        
        # Determine points allowed by each team
        if fav_team == winner:
            fav_points_allowed = pts_l
            und_points_allowed = pts_w
        else:
            fav_points_allowed = pts_w
            und_points_allowed = pts_l
        
        # Initialize or reset team stats
        for team in [fav_team, und_team]:
            if team not in team_stats or team_stats[team]['season'] != season:
                team_stats[team] = {'total_points_allowed': 0, 'game_count': 0, 'season': season}
        
        # Update stats
        team_stats[fav_team]['total_points_allowed'] += fav_points_allowed
        team_stats[fav_team]['game_count'] += 1
        team_stats[und_team]['total_points_allowed'] += und_points_allowed
        team_stats[und_team]['game_count'] += 1
        
        # Calculate and store running averages
        df.at[index, 'fav_def_points_running_avg'] = round(team_stats[fav_team]['total_points_allowed'] / team_stats[fav_team]['game_count'], 2)
        df.at[index, 'und_def_points_running_avg'] = round(team_stats[und_team]['total_points_allowed'] / team_stats[und_team]['game_count'], 2)
    
    return df

# Apply the function to your dataframe
df = calculate_defensive_points_running_average(df)

In [17]:
# Defensive Rating for both Favorite and Underdog

# Favorite
df['def_rating_fav'] = (df['CB_fav']+df['DI_fav']+df['Edge_fav']+
                             df['LB_fav']+df['S_fav'])/5
# Underdog
df['def_rating_und'] = (df['CB_und']+df['DI_und']+df['Edge_und']+
                             df['LB_und']+df['S_und'])/5

# Need to round these to 2 decimal places
df['def_rating_fav'] = df['def_rating_fav'].round(2)
df['def_rating_und'] = df['def_rating_und'].round(2)


# Situational and Team Performance

In [18]:
# Marking if a given game is a divisional game

df['div_game'] = df.apply(lambda x: 1 if x['team_division_fav'] == x['team_division_und'] else 0, axis=1)


In [19]:
# Win Percentages for both Favorite and Underdog

def calculate_win_percentages(df):
    team_stats = {}
    
    df['fav_win_pct'] = 0.0
    df['und_win_pct'] = 0.0
    
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        season = row['season']
        
        # Initialize or reset team stats for new season
        for team in [fav_team, und_team]:
            if team not in team_stats or team_stats[team]['season'] != season:
                team_stats[team] = {'wins': 0, 'games': 0, 'season': season}
        
        # Update stats
        team_stats[fav_team]['games'] += 1
        team_stats[und_team]['games'] += 1
        if fav_team == winner:
            team_stats[fav_team]['wins'] += 1
        else:
            team_stats[und_team]['wins'] += 1
        
        # Calculate and store win percentages
        df.at[index, 'fav_win_pct'] = round(team_stats[fav_team]['wins'] / team_stats[fav_team]['games'], 3)
        df.at[index, 'und_win_pct'] = round(team_stats[und_team]['wins'] / team_stats[und_team]['games'], 3)
    
    return df

# Apply the functions to your dataframe
df = calculate_win_percentages(df)

In [20]:
# Calculating the win percentage over the last 5 games for each team

def calculate_last_5_win_percentages(df):
    team_stats = {}
    
    df['fav_last_5_win_pct'] = 0.0
    df['und_last_5_win_pct'] = 0.0
    
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        season = row['season']
        
        # Initialize or reset team stats for new season
        for team in [fav_team, und_team]:
            if team not in team_stats or team_stats[team]['season'] != season:
                team_stats[team] = {'last_5_results': [], 'season': season}
        
        # Update last 5 games results
        if fav_team == winner:
            team_stats[fav_team]['last_5_results'].append(1)
            team_stats[und_team]['last_5_results'].append(0)
        else:
            team_stats[fav_team]['last_5_results'].append(0)
            team_stats[und_team]['last_5_results'].append(1)
        
        # Keep only last 5 games
        team_stats[fav_team]['last_5_results'] = team_stats[fav_team]['last_5_results'][-5:]
        team_stats[und_team]['last_5_results'] = team_stats[und_team]['last_5_results'][-5:]
        
        # Calculate and store last 5 games win percentages
        df.at[index, 'fav_last_5_win_pct'] = round(sum(team_stats[fav_team]['last_5_results']) / len(team_stats[fav_team]['last_5_results']), 3)
        df.at[index, 'und_last_5_win_pct'] = round(sum(team_stats[und_team]['last_5_results']) / len(team_stats[und_team]['last_5_results']), 3)
    
    return df

df = calculate_last_5_win_percentages(df)

In [21]:
# Last 3 game average win margin 

def calculate_avg_margin_last_3(df):
    team_stats = {}
    
    df['fav_avg_margin_last_3'] = 0.0
    df['und_avg_margin_last_3'] = 0.0
    
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        pts_w = row['pts_w']
        pts_l = row['pts_l']
        season = row['season']
        
        # Initialize or reset team stats for new season
        for team in [fav_team, und_team]:
            if team not in team_stats or team_stats[team]['season'] != season:
                team_stats[team] = {'last_3_margins': [], 'season': season}
        
        # Calculate margin and update last 3 games margins
        margin = pts_w - pts_l
        if fav_team == winner:
            team_stats[fav_team]['last_3_margins'].append(margin)
            team_stats[und_team]['last_3_margins'].append(-margin)
        else:
            team_stats[fav_team]['last_3_margins'].append(-margin)
            team_stats[und_team]['last_3_margins'].append(margin)
        
        # Keep only last 3 games
        team_stats[fav_team]['last_3_margins'] = team_stats[fav_team]['last_3_margins'][-3:]
        team_stats[und_team]['last_3_margins'] = team_stats[und_team]['last_3_margins'][-3:]
        
        # Calculate and store average margin of last 3 games
        if len(team_stats[fav_team]['last_3_margins']) > 0:
            df.at[index, 'fav_avg_margin_last_3'] = round(sum(team_stats[fav_team]['last_3_margins']) / len(team_stats[fav_team]['last_3_margins']), 2)
        if len(team_stats[und_team]['last_3_margins']) > 0:
            df.at[index, 'und_avg_margin_last_3'] = round(sum(team_stats[und_team]['last_3_margins']) / len(team_stats[und_team]['last_3_margins']), 2)
    
    return df

# Apply the function to your dataframe
df = calculate_avg_margin_last_3(df)

In [22]:
# Home & Away win percentage for each team

import pandas as pd

def calculate_home_away_win_percentages(df):
    team_stats = {}
    
    df['fav_home_win_pct'] = 0.0
    df['fav_away_win_pct'] = 0.0
    df['und_home_win_pct'] = 0.0
    df['und_away_win_pct'] = 0.0
    
    df = df.sort_values(['season', 'week'])
    
    for index, row in df.iterrows():
        fav_team = row['favorite']
        und_team = row['underdog']
        winner = row['winner']
        fav_home = row['fav_home']
        season = row['season']
        
        # Initialize or reset team stats for new season
        for team in [fav_team, und_team]:
            if team not in team_stats or team_stats[team]['season'] != season:
                team_stats[team] = {
                    'home_wins': 0, 'home_games': 0,
                    'away_wins': 0, 'away_games': 0,
                    'season': season
                }
        
        # Update stats
        if fav_home:
            team_stats[fav_team]['home_games'] += 1
            team_stats[und_team]['away_games'] += 1
            if fav_team == winner:
                team_stats[fav_team]['home_wins'] += 1
            else:
                team_stats[und_team]['away_wins'] += 1
        else:
            team_stats[fav_team]['away_games'] += 1
            team_stats[und_team]['home_games'] += 1
            if fav_team == winner:
                team_stats[fav_team]['away_wins'] += 1
            else:
                team_stats[und_team]['home_wins'] += 1
        
        # Calculate and store win percentages
        if team_stats[fav_team]['home_games'] > 0:
            df.at[index, 'fav_home_win_pct'] = round(team_stats[fav_team]['home_wins'] / team_stats[fav_team]['home_games'], 3)
        if team_stats[fav_team]['away_games'] > 0:
            df.at[index, 'fav_away_win_pct'] = round(team_stats[fav_team]['away_wins'] / team_stats[fav_team]['away_games'], 3)
        if team_stats[und_team]['home_games'] > 0:
            df.at[index, 'und_home_win_pct'] = round(team_stats[und_team]['home_wins'] / team_stats[und_team]['home_games'], 3)
        if team_stats[und_team]['away_games'] > 0:
            df.at[index, 'und_away_win_pct'] = round(team_stats[und_team]['away_wins'] / team_stats[und_team]['away_games'], 3)
    
    return df

# Apply the function to your dataframe
df = calculate_home_away_win_percentages(df)

In [23]:
# Want to clean some of the columns and there values

# Need to round these to 2 decimal places
df['fav_win_pct'] = df['fav_win_pct'].round(2)
df['und_win_pct'] = df['und_win_pct'].round(2)
df['fav_last_5_win_pct'] = df['fav_last_5_win_pct'].round(2)
df['und_last_5_win_pct'] = df['und_last_5_win_pct'].round(2)
df['fav_home_win_pct'] = df['fav_home_win_pct'].round(2)
df['fav_away_win_pct'] = df['fav_away_win_pct'].round(2)
df['und_home_win_pct'] = df['und_home_win_pct'].round(2)
df['und_away_win_pct'] = df['und_away_win_pct'].round(2)

In [24]:
df.head()

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,C_fav,CB_fav,DI_fav,Edge_fav,LB_fav,LG_fav,LT_fav,QB_fav,RB_fav,RG_fav,RT_fav,S_fav,TE_fav,WR_fav,C_und,CB_und,DI_und,Edge_und,LB_und,LG_und,LT_und,QB_und,RB_und,RG_und,RT_und,S_und,TE_und,WR_und,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,team_division_fav,team_division_und,fav_running_avg,und_running_avg,fav_turnover_running_avg,und_turnover_running_avg,fav_yards_running_avg,und_yards_running_avg,off_rating_fav,off_rating_und,fav_def_yards_running_avg,und_def_yards_running_avg,fav_def_turnovers_running_avg,und_def_turnovers_running_avg,fav_def_points_running_avg,und_def_points_running_avg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_margin_last_3,und_avg_margin_last_3,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,66.0,64.27,73.70,55.75,59.30,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.1,74.73,67.3,67.30,65.53,75.35,67.20,59.1,75.1,82.4,74.05,53.4,59.4,69.60,56.60,71.07,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,AFC West,AFC South,34.0,20.0,0.0,1.0,369.0,360.0,75.55,66.49,360.0,369.0,1.0,0.0,20.0,34.0,65.29,69.00,0,1.0,0.0,1.0,0.0,14.0,-14.0,1.0,0.0,0.0,0.0
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,72.1,60.30,79.53,69.50,69.60,87.7,79.7,75.8,62.95,66.6,53.0,65.35,73.1,77.27,56.9,79.33,65.10,59.60,75.60,60.3,74.1,92.0,77.90,79.3,63.6,68.10,73.25,73.63,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,NFC South,NFC West,25.0,38.0,2.0,0.0,506.0,383.0,72.02,72.33,383.0,506.0,0.0,2.0,38.0,25.0,68.86,69.55,0,0.0,1.0,0.0,1.0,-13.0,13.0,0.0,0.0,0.0,1.0
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,68.7,71.93,74.53,66.85,78.60,63.8,89.4,90.1,81.25,63.6,73.6,73.35,88.5,63.37,72.0,65.47,71.60,83.45,59.67,74.2,88.7,73.5,82.70,56.7,80.5,68.10,72.30,73.45,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,AFC North,AFC North,38.0,6.0,1.0,3.0,381.0,306.0,75.81,74.89,306.0,381.0,3.0,1.0,6.0,38.0,73.05,69.66,1,1.0,0.0,1.0,0.0,32.0,-32.0,1.0,0.0,0.0,0.0
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,67.3,60.20,61.85,68.50,57.03,55.8,73.1,64.1,80.30,63.1,51.8,75.85,60.2,75.43,72.0,69.73,68.17,70.00,73.75,56.1,83.0,63.2,73.80,65.6,60.6,81.25,68.15,75.90,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,AFC East,AFC East,27.0,17.0,2.0,2.0,404.0,254.0,65.68,68.71,254.0,404.0,2.0,2.0,17.0,27.0,64.69,72.58,1,1.0,0.0,1.0,0.0,10.0,-10.0,1.0,0.0,0.0,0.0
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,71.0,59.33,68.45,63.85,65.70,74.0,64.9,79.9,80.05,61.8,69.1,73.60,83.2,72.33,63.4,59.70,87.05,74.20,58.37,58.6,79.3,73.0,80.60,63.6,76.2,77.53,51.80,71.20,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,AFC West,NFC South,34.0,30.0,0.0,0.0,372.0,388.0,72.92,68.63,388.0,372.0,0.0,0.0,30.0,34.0,66.19,71.37,0,1.0,0.0,1.0,0.0,4.0,-4.0,0.0,1.0,0.0,0.0


## Data Checking

Here I am just going to check a couple of instances for different features I created to make sure the data is valid.

In [25]:
df[(df['favorite'] == 'Kansas City Chiefs') | (df['underdog'] == 'Kansas City Chiefs')]

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,C_fav,CB_fav,DI_fav,Edge_fav,LB_fav,LG_fav,LT_fav,QB_fav,RB_fav,RG_fav,RT_fav,S_fav,TE_fav,WR_fav,C_und,CB_und,DI_und,Edge_und,LB_und,LG_und,LT_und,QB_und,RB_und,RG_und,RT_und,S_und,TE_und,WR_und,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,team_division_fav,team_division_und,fav_running_avg,und_running_avg,fav_turnover_running_avg,und_turnover_running_avg,fav_yards_running_avg,und_yards_running_avg,off_rating_fav,off_rating_und,fav_def_yards_running_avg,und_def_yards_running_avg,fav_def_turnovers_running_avg,und_def_turnovers_running_avg,fav_def_points_running_avg,und_def_points_running_avg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_margin_last_3,und_avg_margin_last_3,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,66.0,64.27,73.70,55.75,59.3,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.1,74.73,67.3,67.30,65.53,75.35,67.20,59.1,75.1,82.4,74.05,53.4,59.4,69.60,56.60,71.07,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,AFC West,AFC South,34.00,20.00,0.00,1.00,369.00,360.00,75.55,66.49,360.00,369.00,1.00,0.00,20.00,34.00,65.29,69.00,0,1.00,0.00,1.00,0.0,14.00,-14.00,1.00,0.00,0.00,0.00
29,Kansas City Chiefs,Los Angeles Chargers,0,23,20,-8.5,47.0,1,0.0,0,2,2020,66.0,64.27,73.70,55.75,59.3,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.1,74.73,58.2,70.83,64.35,82.15,67.17,51.7,63.3,47.0,84.80,63.9,77.1,72.75,73.20,75.47,Kansas City Chiefs,Los Angeles Chargers,23.0,20.0,414.0,0.0,479.0,1.0,AFC West,AFC West,28.50,18.00,0.00,0.50,391.50,420.50,75.55,66.07,419.50,354.50,1.00,1.00,20.00,18.00,65.29,71.45,1,1.00,0.50,1.00,0.5,8.50,0.00,1.00,1.00,0.00,1.00
47,Baltimore Ravens,Kansas City Chiefs,1,20,34,-3.5,55.0,0,0.0,0,3,2020,68.7,71.93,74.53,66.85,78.6,63.8,89.4,90.1,81.25,63.6,73.6,73.35,88.5,63.37,66.0,64.27,73.70,55.75,59.30,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.10,74.73,Kansas City Chiefs,Baltimore Ravens,34.0,20.0,517.0,1.0,228.0,1.0,AFC North,AFC West,30.33,30.33,0.67,0.33,338.67,433.33,75.81,75.55,375.67,355.67,2.00,1.00,18.67,20.00,73.05,65.29,0,0.67,1.00,0.67,1.0,11.67,10.33,0.50,1.00,1.00,1.00
61,Kansas City Chiefs,New England Patriots,1,26,10,-11.0,48.5,1,1.0,0,4,2020,66.0,64.27,73.70,55.75,59.3,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.1,74.73,74.3,75.13,66.63,72.75,68.65,79.2,70.7,57.8,71.50,75.1,70.1,68.20,79.00,65.00,Kansas City Chiefs,New England Patriots,26.0,10.0,323.0,1.0,357.0,4.0,AFC West,AFC East,29.25,24.25,0.50,1.75,405.75,396.00,75.55,71.41,356.00,349.00,1.75,2.00,17.50,23.00,65.29,70.27,0,1.00,0.50,1.00,0.5,11.00,-1.67,1.00,1.00,1.00,0.00
67,Kansas City Chiefs,Las Vegas Raiders,1,32,40,-11.0,54.5,0,0.0,1,5,2020,66.0,64.27,73.70,55.75,59.3,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.1,74.73,71.0,59.33,68.45,63.85,65.70,74.0,64.9,79.9,80.05,61.8,69.1,73.60,83.20,72.33,Las Vegas Raiders,Kansas City Chiefs,40.0,32.0,490.0,1.0,413.0,1.0,AFC West,AFC West,29.80,30.20,0.60,1.40,407.20,399.40,75.55,72.92,382.80,393.60,1.60,0.60,22.00,30.40,65.29,66.19,1,0.80,0.60,0.80,0.6,7.33,-5.00,0.67,1.00,0.50,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,Kansas City Chiefs,Buffalo Bills,1,17,20,-1.5,49.0,0,0.0,0,14,2023,89.9,71.27,64.65,64.83,72.7,78.4,59.3,92.0,66.85,72.3,59.5,64.90,91.3,70.17,61.0,63.77,70.55,76.97,62.75,53.5,73.8,91.8,73.20,61.4,53.4,67.05,78.35,67.80,Buffalo Bills,Kansas City Chiefs,20.0,17.0,327.0,1.0,346.0,2.0,AFC West,AFC East,22.46,26.77,1.69,1.62,361.31,378.92,75.52,68.25,299.92,321.92,1.15,1.77,17.54,18.77,67.67,68.22,0,0.62,0.54,0.40,0.4,1.00,8.67,0.50,0.71,0.83,0

### Checking Nan

Here I will be double checking to see if Nan values exist.

In [26]:
nan_columns = df.isna().sum()

nan_columns = nan_columns[nan_columns > 0]

nan_columns

Series([], dtype: int64)

As you can see we have no Nan columns which is great!

### Extreme/Outliers

Here I will be checking to make sure that we do not have any super large (or small) values for each of the created columns.

In [28]:
df.describe()

,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,C_fav,CB_fav,DI_fav,Edge_fav,LB_fav,LG_fav,LT_fav,QB_fav,RB_fav,RG_fav,RT_fav,S_fav,TE_fav,WR_fav,C_und,CB_und,DI_und,Edge_und,LB_und,LG_und,LT_und,QB_und,RB_und,RG_und,RT_und,S_und,TE_und,WR_und,pts_w,pts_l,yds_w,to_w,yds_l,to_l,fav_running_avg,und_running_avg,fav_turnover_running_avg,und_turnover_running_avg,fav_yards_running_avg,und_yards_running_avg,off_rating_fav,off_rating_und,fav_def_yards_running_avg,und_def_yards_running_avg,fav_def_turnovers_running_avg,und_def_turnovers_running_avg,fav_def_points_running_avg,und_def_points_running_avg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_margin_last_3,und_avg_margin_last_3,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct
count,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.00000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.00000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.00000,1071.000000,1071.000000
mean,0.602241,25.261438,20.393091,-5.305322,45.297386,0.660131,0.471522,0.461251,9.380019,2021.521942,69.093744,67.579720,69.013641,72.868796,64.954034,68.179272,77.471989,78.269468,75.909057,68.467320,78.319421,70.270187,71.467600,74.397824,67.431092,66.436088,68.685415,72.187666,63.467572,65.226891,75.438189,74.745938,74.478431,67.404388,96.24267,69.112577,66.662979,73.376639,28.354809,17.299720,370.632120,0.855275,316.049486,1.773109,24.818273,21.086228,1.252222,1.355070,361.744949,331.409916,72.914725,70.67479,337.360289,355.892904,1.387880,1.223651,21.778198,24.141363,68.937283,67.977628,0.358543,0.596424,0.404332,0.605742,0.393744,3.493604,-3.501513,0.595294,0.55718,0.404687,0.373501
std,0.489664,9.804672,9.510521,3.499087,4.672252,0.473886,0.499422,0.498729,5.230959,1.112056,11.394113,5.772155,7.634786,7.994634,8.924119,11.229405,9.454922,10.791806,6.906856,10.926431,103.510675,6.108172,10.259608,5.006858,10.824522,6.685272,9.913088,7.524740,9.045469,10.962455,10.260308,10.708959,7.702054,11.205864,221.88785,7.138283,9.089051,5.074359,8.423442,8.143961,77.528741,0.898892,81.159370,1.214348,5.071645,4.905678,0.524011,0.496069,43.454854,44.236542,3.844404,3.62448,45.338685,40.423492,0.527142,0.514384,4.816106,4.513756,3.778488,4.305925,0.479797,0.230719,0.236234,0.266420,0.268492,8.796022,8.742943,0.306095,0.30034,0.295296,0.299818
min,0.000000,0.000000,0.000000,-20.000000,28.500000,0.000000,0.000000,0.000000,1.000000,2020.000000,23.400000,49.930000,41.150000,52.950000,36.050000,37.700000,52.300000,47.000000,59.100000,32.200000,41.800000,55.250000,48.200000,60.300000,23.400000,49.930000,37.900000,53.850000,36.050000,37.700000,52.300000,47.000000,58.900000,32.200000,41.80000,52.400000,48.200000,60.300000,3.000000,0.000000,145.000000,0.000000,47.000000,0.000000,3.000000,0.000000,0.000000,0.000000,142.000000,171.000000,63.610000,63.61000,163.670000,142.000000,0.000000,0.000000,0.000000,3.000000,57.460000,57.350000,0.000000,0.000000,0.000000,0.000000,0.000000,-35.000000,-40.000000,0.000000,0.00000,0.000000,0.000000
25%,0.000000,19.000000,14.000000,-7.000000,42.000000,0.000000,0.000000,0.000000,5.000000,2021.000000,62.650000,63.930000,64.650000,67.550000,59.050000,61.500000,71.900000,70.900000,71.100000,61.400000,66.700000,66.850000,63.500000,7

In [29]:
# We can see that the RT for the Jets in 2023 has a rating of 2021.0

df['RT_fav'] = df['RT_fav'].replace(2021.0, 75.0)
df['RT_und'] = df['RT_und'].replace(2021.0, 75.0)


In [30]:
df.describe()

,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,C_fav,CB_fav,DI_fav,Edge_fav,LB_fav,LG_fav,LT_fav,QB_fav,RB_fav,RG_fav,RT_fav,S_fav,TE_fav,WR_fav,C_und,CB_und,DI_und,Edge_und,LB_und,LG_und,LT_und,QB_und,RB_und,RG_und,RT_und,S_und,TE_und,WR_und,pts_w,pts_l,yds_w,to_w,yds_l,to_l,fav_running_avg,und_running_avg,fav_turnover_running_avg,und_turnover_running_avg,fav_yards_running_avg,und_yards_running_avg,off_rating_fav,off_rating_und,fav_def_yards_running_avg,und_def_yards_running_avg,fav_def_turnovers_running_avg,und_def_turnovers_running_avg,fav_def_points_running_avg,und_def_points_running_avg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_margin_last_3,und_avg_margin_last_3,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct
count,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.00000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.00000,1071.000000,1071.000000
mean,0.602241,25.261438,20.393091,-5.305322,45.297386,0.660131,0.471522,0.461251,9.380019,2021.521942,69.093744,67.579720,69.013641,72.868796,64.954034,68.179272,77.471989,78.269468,75.909057,68.467320,72.868441,70.270187,71.467600,74.397824,67.431092,66.436088,68.685415,72.187666,63.467572,65.226891,75.438189,74.745938,74.478431,67.404388,70.804762,69.112577,66.662979,73.376639,28.354809,17.299720,370.632120,0.855275,316.049486,1.773109,24.818273,21.086228,1.252222,1.355070,361.744949,331.409916,72.914725,70.67479,337.360289,355.892904,1.387880,1.223651,21.778198,24.141363,68.937283,67.977628,0.358543,0.596424,0.404332,0.605742,0.393744,3.493604,-3.501513,0.595294,0.55718,0.404687,0.373501
std,0.489664,9.804672,9.510521,3.499087,4.672252,0.473886,0.499422,0.498729,5.230959,1.112056,11.394113,5.772155,7.634786,7.994634,8.924119,11.229405,9.454922,10.791806,6.906856,10.926431,10.169535,6.108172,10.259608,5.006858,10.824522,6.685272,9.913088,7.524740,9.045469,10.962455,10.260308,10.708959,7.702054,11.205864,10.947184,7.138283,9.089051,5.074359,8.423442,8.143961,77.528741,0.898892,81.159370,1.214348,5.071645,4.905678,0.524011,0.496069,43.454854,44.236542,3.844404,3.62448,45.338685,40.423492,0.527142,0.514384,4.816106,4.513756,3.778488,4.305925,0.479797,0.230719,0.236234,0.266420,0.268492,8.796022,8.742943,0.306095,0.30034,0.295296,0.299818
min,0.000000,0.000000,0.000000,-20.000000,28.500000,0.000000,0.000000,0.000000,1.000000,2020.000000,23.400000,49.930000,41.150000,52.950000,36.050000,37.700000,52.300000,47.000000,59.100000,32.200000,41.800000,55.250000,48.200000,60.300000,23.400000,49.930000,37.900000,53.850000,36.050000,37.700000,52.300000,47.000000,58.900000,32.200000,41.800000,52.400000,48.200000,60.300000,3.000000,0.000000,145.000000,0.000000,47.000000,0.000000,3.000000,0.000000,0.000000,0.000000,142.000000,171.000000,63.610000,63.61000,163.670000,142.000000,0.000000,0.000000,0.000000,3.000000,57.460000,57.350000,0.000000,0.000000,0.000000,0.000000,0.000000,-35.000000,-40.000000,0.000000,0.00000,0.000000,0.000000
25%,0.000000,19.000000,14.000000,-7.000000,42.000000,0.000000,0.000000,0.000000,5.000000,2021.000000,62.650000,63.930000,64.650000,67.550000,59.050000,61.500000,71.900000,70.900000,71.100000,61.400000,66.700000,66.850000,63.500000

Our data looks mostly valid. There are no crazy outliers (in terms of things that did happen). 

For example, 726 yards might be an outlier, but the Doplhins really did that to the Broncos in 2023.

# Write to CSV

In [31]:
fp = ('/Users/epainter/Desktop/bet_model_v2/data/processing/fc_1.csv')

df.to_csv(fp, index=False)

print(f"Data saved to {fp}")

Data saved to /Users/epainter/Desktop/bet_model_v2/data/processing/fc_1.csv
